In [1]:
from ISC_utils import*

In [3]:
path=('')# path to file
oxygens =['0','2','3','4','5','7','12','21']
files =[''] #files

data=anaylsis(path, oxygens, files)

c:\Users\will6605\Documents\vscode\.venv\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\will6605\Documents\vscode\.venv\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
clusters=[3,3,3,2,2,3,1,1]
data2=data.copy(deep=True)
data2['oxy'] = data2['oxy'].astype(int)
data3, sort, SO2, LPF, polys, solys, SO2s, SO2p, p50, SO2sstd, SO2pstd, num_polys, num_solys = fractions(data2, clusters) 

In [ ]:
solystrain=strain(solys,[6e-7,1e-6])
df1=pd.DataFrame(solystrain["params"].tolist()).add_prefix("c")
solystrain = pd.concat([solystrain.reset_index(), df1], axis=1)
solystrain=solystrain.dropna()
solystrain.groupby('oxy', as_index=False)['c0'].median()
polystrain=strain(polys,[6e-3,1e-6])
df2=pd.DataFrame(polystrain["params"].tolist()).add_prefix("c")
polystrain = pd.concat([polystrain.reset_index(), df2], axis=1)
polystrain=polystrain.dropna()

In [ ]:
LABELS = ["poly", "soly"] #these are the two labels to classify to

IMG_SIZE = 100 #image isze is 100x100

model=keras.models.load_model('CNN/model5.h5', safe_mode=False) #load the model

In [ ]:
count=1
threshold=0.5 #threshol is set at 0.5 to start
solystrain['pred'] = np.nan #set the predicitin column to nan to start
oxy=[0,2,3,4,5,7,12,21] # List you oxygen tensions
for i in range(len(oxy)): #run through the list of oxygen tensions
    df = solystrain.loc[solystrain['oxy'] == oxy[i]] #make a dataframe of each individual O2 (helps with speed)
    for index,c in df.iterrows(): #run through the dataframe
        img = np.zeros([100,100,3]) #CNN was trained on 3d images
        img[:,:,0] = c.cell_img3
        img[:,:,1] = c.cell_img3
        img[:,:,2] = c.cell_img3
        new_img = cv.resize(img, (IMG_SIZE, IMG_SIZE))
        new_shape = new_img.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
        predictions = model.predict(new_shape) #this is the line that calls the CNN to classify
        print(predictions) #print the %prediciton
        print(LABELS[np.argmax(predictions)]) #Print the accosicated label
        # sort.label[index]=(np.argmax(np.where(predictions > threshold, 1, 0))) #add the correct label to the origina dataframe
        solystrain.pred[index]=predictions[0][1] #add the correct prediction % to dataframe (you can use this column to adjust threshold without rerunning CNN)
        if predictions[0][1]>threshold:
            solystrain.label[index]=1
        else:
            solystrain.label[index]=0
        # print(count)
        count +=1
